In [75]:
import torch
import torch.optim as optim

In [76]:
# 재실행해도 다음에도 같은 결과가 나오도록 랜덤 시드(random seed)를 수동으로 설정
torch.manual_seed(1)

# Linear Regression(선형 회귀)

## 1. Data Definition

- `training dataset` : 예측을 위해 사용하는 데이터(학습용 데이터)
- `test dataset` : 모델이 얼마나 잘 작동하는지 판별하는 데이터

`PyTorch`는 `training dataset`이 `torch.tensor` 형태를 가지고 있어야 함

입력과 출력을 각기 다른 텐서에 저장할 필요가 있음

보편적으로 입력은 `x`, 출력은 `y`를 사용하여 표기함


$ X_{train} = \begin{pmatrix} 1\\2\\3 \end{pmatrix} $
$ Y_{trian} = \begin{pmatrix} 2\\4\\6 \end{pmatrix} $ 

In [77]:
# 변수 선언
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [78]:
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [79]:
print(y_train)
print(y_train.shape)

tensor([[2.],
        [4.],
        [6.]])
torch.Size([3, 1])


## 2. 가설(Hypothesis) 수립

`Hypothesis(가설)` : `Machine Learning`에서 식을 세울때 세운 식, 임의로 추측하여 세우거나, 경험적으로 알고 있는 식일 수 있음 가설이 아니라고 판단되면 계속 수정해 나가게 됨

선형 회귀의 가설(직선의 방정식)

$ y = Wx + b $

가설의 $H$를 따서 $y$대신 사용하기도 함

$ H(x) = Wx + b $

$W$ 를 `Weight(가중치)`, $b$ 를 `bias(편향)` 이라 함

In [80]:
# 가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
W = torch.zeros(1, requires_grad=True) 
# 가중치 W를 출력
print(W)

# 편향 b도 0으로 초기화하고, 학습을 통해 값이 변경되는 변수임을 명시
b = torch.zeros(1, requires_grad=True)
print(b)

# W,b 둘다 0이므로 현 방정식은 y = 0 * x + 0
# requires_grad 속성을 True로 설정하면 자동 미분 기능이 적용
# 적용된 텐서에 연산을 하면, 계산 그래프가 생성되며 backward 함수를 호출하면 그래프로부터 자동으로 미분이 계산됨

tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)


In [81]:
# 직선의 방정식에 해당되는 가설을 선언
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## 3. 비용 함수(Cost function)

`비용 함수(cost function)` = `손실 함수(loss function)` = `오차 함수(error function)` = `목적 함수(objective function)`

이 중 비용 함수(cost function) 와 손실 함수(loss function) 를 가장 많이 사용

`Linear Regression(선형 회귀)`는 위와 같이 직선의 방정식을 지님

데이터를 가장 잘 나타내는 직선의 방적식을 찾는 방법임

`오차(error)`가 가장 적은 직선이 데이터를 가장 잘 나타내는 직선이라 할 수 있음

수식적으로 단순히 `오차 = 실제값 - 예측값`으로 정의하면 오차값이 음수가 나오는 경우가 발생함

`총 오차(total error)`를 구하기 위해, 오차를 그냥 전부 더하는 것이 아니라, 각 오차들을 제곱해준 뒤에 전부 더함.

$ 1\over n $ $ \displaystyle\sum_{ i=1 }^{ n }{ [y^{(i)} - H(x^{(i)})]^2 } $

이를 `평균 제곱 오차(Mean Squared Error, MSE)` 라 함

MSE는 회귀 문제에 적절한 $W$와 $b$를 찾기위해 최적화된 식임, 평균 제곱 오차의 값을 최소값으로 만드는 $W$와 $b$를 찾는 것이 가장 훈련 데이터를 잘 반영한 직선을 찾아내는 일이기 때문

MSE를 $W$와 $b$에 의한 비용 함수(cost function)로 재정의하면 다음과 같음

$ cost(W,b) = $ $ 1\over n $ $ \displaystyle\sum_{ i=1 }^{ n }{ [y^{(i)} - H(x^{(i)})]^2 } $

In [82]:
# 선형 회귀의 비용 함수에 해당되는 평균 제곱 오차를 선언
cost = torch.mean((hypothesis - y_train) ** 2) 
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


## 옵티마이저(Optimizer) - 경사 하강법(Gradient Descent)

`옵티마이저(Optimizer)` 알고리즘 : 최적화 알고리즘, 여기에선 비용 함수(cost Function)의 값을 최소로 하는 $W$와 $b$를 찾는 방법

`학습(training)` : 옵티마이저 알고리즘을 통해 적절한 $W$와 $b$를 찾아내는 과정

$ H(x) = Wx + b $

$ cost(W,b) = $ $ 1\over n $ $ \displaystyle\sum_{ i=1 }^{ n }{ [y^{(i)} - H(x^{(i)})]^2 } $

다음 식에서 가중치 $W$는 기울기, 편향 $b$는 절편과 같음

기울기가 지나치게 크면 실제값과 예측값의 오차가 커지고, 기울기가 지나치게 작아도 실제값과 예측값의 오차가 커짐, 편향 또한 지나치게 크거나 작으면 오차가 커짐

$W$ 와 $cost$ 관계를 x, y로 두고 그래프로 그리면 아래로 볼록한 2차 방정식 그래프를 가짐

$W$ 가 무한대로 커지면 $cost$ 도 무한대로 커지고 $W$ 가 무한대로 작아져도 $cost$도 무한대로 커짐

따라서 $cost$ 가 가장 작은 지점은 맨 아래의 볼록한 지점인 그래프에서 꼭지점의 $W$ 임

이를 찾는 과정에 사용하는 것이 `경사 하강법(Gradient Descent)`

경사 하강법은 미분을 하여 접선의 기울기가 0에 가까운 지점을 찾아나감, $cost$가 최소화가 되는 지점은 접선의 기울기가 0이 되는 지점이며, 또한 미분값이 0이 되는 지점이기 때문

즉 경사 하강법은 Cost function을 미분하여 현재 $W$에서의 접선의 기울기를 구하고, 접선의 기울기가 낮은 방향으로 $W$의 값을 변경하는 작업을 반복하는 것에 있음

$ F = \frac{ \partial cost(W) }{ \partial W } $

- 기울기 $F$가 음수일때: $W$값이 증가
- 기울기 $F$가 양수일때: $W$값이 감소

$ W := W - \alpha \frac{ \partial }{ \partial W }cost(W) $

$\alpha$는 `학습률(learning rate)`, $W$ 값을 변경할 때, 얼마나 크게 변경할지를 결정 또는 그래프의 한 점으로보고 접선의 기울기가 0일 때까지 경사를 따라 내려간다는 관점에서는 얼마나 큰 폭으로 이동할지를 결정

학습률 $\alpha$를 무작정 크게 한다고 $W$를 빠르게 찾는 것은 아님, 오히려 지나치게 높으면 $cost$값이 발산하는 상황이 발생

반대로 학습률 $\alpha$가 지나치게 낮으면 학습속도가 느려짐

따라서 적절한 $\alpha$를 찾아내는게 중요

가설, 비용 함수, 옵티마이저는 머신 러닝 분야에서 사용되는 포괄적 개념

풀고자하는 각 문제에 따라 가설, 비용 함수, 옵티마이저는 전부 다를 수 있음

선형 회귀에 가장 적합한 비용 함수는 평균 제곱 오차, 옵티마이저는 경사 하강법

In [83]:
# 경사 하강법을 구현합니다. 아래의 'SGD'는 Stochastic Gradient Descent 경사 하강법의 일종입니다. lr은 학습률(learning rate)를 의미합니다. 학습 대상인 W와 b가 SGD의 입력됨
optimizer = optim.SGD([W, b], lr=0.01)

In [84]:
# gradient를 0으로 초기화
optimizer.zero_grad() 
# 비용 함수를 미분하여 gradient 계산
cost.backward() 
# W와 b를 업데이트
optimizer.step() 

In [85]:
nb_epochs = 2000 # 경사하강법을 반복할 횟수
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad() # 파이토치는 미분을 통해 얻은 기울기를 이전에 계산된 기울기 값에 누적함, 미분값을 계속 0으로 초기화시켜줘야 됨
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} \
              W: {W.item():.3f}, \
              b: {b.item():.3f}, \
              Cost: {cost.item():.6f}')

Epoch    0/2000               W: 0.353,               b: 0.151,               Cost: 14.770963
Epoch  100/2000               W: 1.746,               b: 0.577,               Cost: 0.047939
Epoch  200/2000               W: 1.801,               b: 0.453,               Cost: 0.029624
Epoch  300/2000               W: 1.843,               b: 0.356,               Cost: 0.018306
Epoch  400/2000               W: 1.877,               b: 0.280,               Cost: 0.011312


Epoch  500/2000               W: 1.903,               b: 0.220,               Cost: 0.006990
Epoch  600/2000               W: 1.924,               b: 0.173,               Cost: 0.004319
Epoch  700/2000               W: 1.940,               b: 0.136,               Cost: 0.002669
Epoch  800/2000               W: 1.953,               b: 0.107,               Cost: 0.001649
Epoch  900/2000               W: 1.963,               b: 0.084,               Cost: 0.001019
Epoch 1000/2000               W: 1.971,               b: 0.066,               Cost: 0.000630
Epoch 1100/2000               W: 1.977,               b: 0.052,               Cost: 0.000389
Epoch 1200/2000               W: 1.982,               b: 0.041,               Cost: 0.000240
Epoch 1300/2000               W: 1.986,               b: 0.032,               Cost: 0.000149
Epoch 1400/2000               W: 1.989,               b: 0.025,               Cost: 0.000092
Epoch 1500/2000               W: 1.991,               b: 0.020,       

최종 훈련 결과를 보면 최적의 기울기 $W$는 2에 가깝고, $b$는 0에 가까움

현재 훈련 데이터가 x_train은 [[1], [2], [3]]이고 y_train은 [[2], [4], [6]]인 것을 감안하면

실제 정답은 $W$가 2이고, $b$가 0인 $H(x) = 2x$ 이므로 거의 정답에 가깝게 학습됨